In [48]:
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from ipywidgets import interactive, FloatSlider
import ipywidgets as widgets

In [49]:
# Define DH parameters
d1, a1, alpha1 = 0.1, 0, math.pi / 2
d2, a2, alpha2 = 0, 0.5, 0
d3, a3, alpha3 = 0, 0.5, 0
d4, a4, alpha4 = 0, 0, math.pi / 2
d5, a5, alpha5 = 0.1, 0, 0

In [50]:

def inverse_kinematics(Px, Py, Pz, d1, a2, a3, d5, omega):
    # Calculate wrist position coordinates
    R = d5 * math.cos(math.radians(omega))
    theta1 = math.degrees(math.atan2(Py, Px))
    theta1_rad = math.radians(theta1)

    Pxw = Px - R * math.cos(theta1_rad)
    Pyw = Py - R * math.sin(theta1_rad)
    Pzw = Pz - d5 * math.sin(math.radians(omega))

    # Calculate Rw and S
    Rw = math.sqrt(Pxw**2 + Pyw**2)
    S = math.sqrt((Pzw - d1) ** 2 + Rw**2)

    # Calculate theta2 and theta3
    alpha = math.degrees(math.atan2(Pzw - d1, Rw))
    beta = math.degrees(math.acos((a2**2 + S**2 - a3**2) / (2 * a2 * S)))

    theta2 = alpha + beta
    # or
    theta2_alt = alpha - beta

    # Calculate theta3

    theta3 = math.degrees(math.acos((S**2 - a2**2 - a3**2) / (2 * a2 * a3)))

    # or
    theta3 = -theta3  # Adjust for proper direction

    # Calculate theta4
    theta234 = 90 - omega
    # theta234_alt = 90 + omega
    theta4 = theta234 - theta2 - theta3

    return theta1, theta2, theta3, theta4, theta234


In [55]:

def forward_kinematics(d1, a2, a3, d5, theta1, theta2, theta3, theta4):
    # Convert angles to radians
    theta1 = math.radians(theta1)
    theta2 = math.radians(theta2)
    theta3 = math.radians(theta3)
    theta4 = math.radians(theta4)

    # Joint positions
    x0, y0, z0 = 0, 0, 0
    x1, y1, z1 = 0, 0, d1
    x2 = a2 * math.cos(theta1) * math.cos(theta2)
    y2 = a2 * math.sin(theta1) * math.cos(theta2)
    z2 = d1 + a2 * math.sin(theta2)
    x3 = x2 + a3 * math.cos(theta1) * math.cos(theta2 + theta3)
    y3 = y2 + a3 * math.sin(theta1) * math.cos(theta2 + theta3)
    z3 = z2 + a3 * math.sin(theta2 + theta3)

    x4 = x3 + a4 * math.cos(theta1) * math.cos(theta2 + theta3 + theta4)
    y4 = y3  + a4 * math.sin(theta1) * math.cos(theta2 + theta3 + theta4)
    z4 = z3 + a4 * math.sin(theta2 + theta3 + theta4)

    

    return [(x0, y0, z0), (x1, y1, z1), (x2, y2, z2), (x3, y3, z3), (x4, y4, z4)]


In [52]:

def plot_robot(joint_positions):
    # Unpack joint positions
    x, y, z = zip(*joint_positions)

    # Plot the robot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    ax.plot(x, y, z, "o-", markersize=10, label="Robot Arm")
    ax.scatter(x, y, z, c="k")

    # Add text labels for the links and lengths
    for i in range(len(joint_positions) - 1):
        ax.text(
            (x[i] + x[i + 1]) / 2,
            (y[i] + y[i + 1]) / 2,
            (z[i] + z[i + 1]) / 2,
            f"Link {i + 1}",
            color="black",
        )

    # Set labels and title
    ax.set_xlabel("X axis")
    ax.set_ylabel("Y axis")
    ax.set_zlabel("Z axis")
    ax.set_title("3D Robot Configuration")
    ax.legend()
    ax.set_box_aspect([1, 1, 1])
   # plt.show()



In [53]:

def update_robot(Px, Py, Pz, omega):
    theta1, theta2, theta3, theta4 = inverse_kinematics(Px, Py, Pz, d1, a2, a3, d5, omega)
    joint_positions = forward_kinematics(d1, a2, a3, d5, theta1, theta2, theta3, theta4)
    end_effector_position = joint_positions[-1]
    plot_robot(joint_positions)
    print(f"Theta1: {theta1:.2f} degrees")
    print(f"Theta2: {theta2:.2f} degrees")
    print(f"Theta3: {theta3:.2f} degrees")
    print(f"Theta4: {theta4:.2f} degrees")
    print(f"End Effector Position: {end_effector_position}")
    
    plot_robot(joint_positions)

In [54]:
# Interactive sliders
Px_slider = FloatSlider(value=0.8, min=-1.1, max=1.1, step=0.01, description='Px')
Py_slider = FloatSlider(value=0, min=-1.1, max=1.1, step=0.01, description='Py')
Pz_slider = FloatSlider(value=-0.4, min=-1.0, max=1.0, step=0.01, description='Pz')
omega_slider = FloatSlider(value=-90, min=-180, max=180, step=1, description='Omega')

interactive_plot = interactive(update_robot, Px=Px_slider, Py=Py_slider, Pz=Pz_slider, omega=omega_slider)
interactive_plot

interactive(children=(FloatSlider(value=0.8, description='Px', max=1.1, min=-1.1, step=0.01), FloatSlider(valu…